Notebook dealing with the final preprocessing steps for the neural networks.


In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.utils.data
import pandas as pd
from torch.utils import data
from numpy import array
from numpy import argmax
import argparse
from torch.autograd import Variable
from torch import optim
import numpy as np
import os
#from sklearn.model_selection import train_test_split
import logging
import pickle as pkl
import warnings
pd.options.mode.chained_assignment = None
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="10"
import glob
import random
random.seed(1991)
#torch.set_default_tensor_type('torch.cuda.DoubleTensor')

In [2]:
torch.cuda.is_available()

True

In [3]:
torch.manual_seed(1991)
if not torch.cuda.is_available():
    
        print("WARNING: You have a CUDA device, so you should probably run with --cuda")

device = torch.device("cuda")

In [16]:
heads=pd.read_csv("/data/dharp/compounds/datasets/v2_aware/heads_reduced.csv",sep="\t",header=None)
#heads=heads.drop(1,axis=1).groupby(0).mean()
#heads.columns=list(range(0,300))
#heads.index.name="head"
#heads.reset_index(inplace=True)
heads=heads.iloc[:,0:102]
heads=pd.pivot_table(heads, index=0, columns=1)
heads.fillna(0,inplace=True)
heads_final_1=heads.columns.get_level_values(0)
heads_final_2=heads.columns.get_level_values(1)
cur_year=0
new_columns=[]
for year in heads_final_2:
    new_columns.append(str(year)+"_"+str(heads_final_1[cur_year]))
    cur_year+=1
heads.columns=new_columns
heads.index.name="head"
heads.reset_index(inplace=True)
heads

,head,1950_2,1960_2,1970_2,1980_2,1950_3,1960_3,1970_3,1980_3,1950_4,...,1970_99,1980_99,1950_100,1960_100,1970_100,1980_100,1950_101,1960_101,1970_101,1980_101
0,#,0.021435,0.025504,0.047466,0.117382,-0.018350,-0.021564,-0.039509,-0.093684,-0.020596,...,0.006726,0.002771,-0.009793,-0.009133,0.009621,0.054433,0.003588,0.008540,0.013667,0.043197
1,%,0.094702,0.128815,0.130665,0.144362,-0.083040,-0.112580,-0.114191,-0.125634,-0.001795,...,-0.031983,-0.030000,0.054993,0.003823,0.048960,0.022864,0.126578,0.130142,0.146766,0.114835
2,',0.077669,0.080465,0.066294,0.160384,-0.070517,-0.074054,-0.060967,-0.148100,-0.105315,...,0.036757,0.021841,0.024167,0.017708,0.021098,0.028915,0.033183,0.007099,-0.001925,0.027003
3,'s,0.018810,0.029860,0.026236,0.030945,-0.017299,-0.028202,-0.024472,-0.028976,-0.020997,...,-0.022153,-0.018242,0.039587,0.034616,0.039676,0.038218,0.054918,0.053769,0.054504,0.056778
4,*,0.150023,0.145956,0.098259,0.084887,-0.139367,-0.134609,-0.090122,-0.077429,-0.181223,...,-0.021123,-0.012461,0.079874,0.068898,0.024323,0.030468,0.044482,0.033783,0.023447,0.032875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22744,®,0.056500,0.075998,0.113051,0.136946,-0.041166,-0.057922,-0.101045,-0.125770,-0.055678,...,0.060843,0.034741,0.044626,-0.043937,-0.000029,0.014644,0.020984,0.042164,0.075830,0.092113
22745,°,0.086092,0.092841,0.077362,0.063651,-0.073901,-0.079384,-0.065524,-0.052992,-0.019014,...,-0.011559,-0.009496,0.093245,0.078669,0.067218,0.056567,-0.015210,-0.021972,-0.038274,-0.040775
22746,±,0.093368,0.075311,0.060565,0.074344,-0.080031,-0.062496,-0.049055,-0.061190,-0.093475,...,-0.031903,-0.020244,0.082588,0.084985,0.074826,0.063165,0.002705,0.003214,0.007560,0.014484
22747,•,0.035314,0.029224,0.031102,0.058320,-0.031566,-0.025343,-0.027240,-0.041351,-0.038273,...,-0.031090,-0.032495,0.060855,0.059146,0.059298,0.058374,0.045317,0.048687,0.052215,0.063697


In [17]:
modifiers=pd.read_csv("/data/dharp/compounds/datasets/v2_aware/modifiers_reduced.csv",sep="\t",header=None)
modifiers=modifiers.iloc[:,0:102]
modifiers=pd.pivot_table(modifiers, index=0, columns=1)
modifiers.fillna(0,inplace=True)
modifiers_final_1=modifiers.columns.get_level_values(0)
modifiers_final_2=modifiers.columns.get_level_values(1)
cur_year=0
new_columns=[]
for year in modifiers_final_2:
    new_columns.append(str(year)+"_"+str(modifiers_final_1[cur_year]))
    cur_year+=1
modifiers.columns=new_columns
modifiers.index.name="modifier"
modifiers.reset_index(inplace=True)
modifiers

,modifier,1950_2,1960_2,1970_2,1980_2,1950_3,1960_3,1970_3,1980_3,1950_4,...,1970_99,1980_99,1950_100,1960_100,1970_100,1980_100,1950_101,1960_101,1970_101,1980_101
0,#,0.053905,0.051248,0.050715,0.126901,-0.050517,-0.048328,-0.048088,-0.103937,-0.002718,...,0.017726,0.086166,-0.021787,-0.024593,-0.025593,0.019520,0.005133,0.000700,-0.000595,0.037029
1,%,0.164367,0.151576,0.153728,0.165798,-0.101373,-0.102792,-0.116771,-0.134395,0.079559,...,0.030487,0.030365,-0.030997,-0.023039,-0.013567,-0.011784,0.014303,0.019033,0.025030,0.029229
2,',0.262998,0.211746,0.107343,0.076527,-0.230006,-0.186894,-0.097087,-0.067757,-0.370949,...,0.007622,0.061487,0.029612,0.033653,0.037385,0.077108,0.002442,0.006050,0.005390,0.019043
3,*,0.078771,0.086963,0.050306,0.035195,-0.043397,-0.064580,-0.044731,-0.031937,-0.062147,...,0.017732,0.008107,-0.006567,-0.025831,-0.006560,-0.015499,0.068787,-0.026305,-0.057286,-0.048006
4,+,0.084418,0.087836,0.092767,0.095474,-0.076887,-0.080986,-0.083382,-0.084759,-0.072252,...,-0.019244,-0.015193,0.050953,0.047647,0.050491,0.045680,0.069288,0.063053,0.058258,0.033322
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21104,°,0.043555,0.029735,0.022878,0.019351,-0.038489,-0.026391,-0.020412,-0.017317,-0.008794,...,-0.003112,-0.003272,0.002441,-0.006434,-0.010024,-0.012058,-0.008350,-0.006610,-0.006105,-0.005376
21105,±,0.106520,0.048854,0.082716,0.098216,-0.098464,-0.042765,-0.072139,-0.084720,-0.125612,...,-0.015172,-0.009217,0.058953,0.020987,0.053364,0.067248,0.005836,0.039834,0.028066,0.013866
21106,•,0.185728,0.163963,0.146292,0.196452,-0.157946,-0.149069,-0.130661,-0.174692,-0.183697,...,0.059420,0.047184,0.053571,0.086248,0.095165,0.070566,0.040782,0.079531,0.121284,0.072652
21107,™,0.026344,0.027223,0.026286,0.026724,-0.026401,-0.027321,-0.026340,-0.026796,-0.045601,...,-0.108516,-0.108238,0.074266,0.074522,0.074250,0.074383,0.047935,0.047248,0.047928,0.047451


In [18]:
with open('/data/dharp/compounds/datasets/compound_testset.txt','r') as f:
    novel_compounds_list=f.read().split("\n")
novel_compounds_list=novel_compounds_list[:-1]
len(novel_compounds_list)

12572

novel_compounds_list = pkl.load( open( "/data/dharp/compounding/datasets/novel_compounds_list.pkl", "rb" ) )
m, h = zip(*novel_compounds_list)
heads_list=list(set(h))
modifiers_list=list(set(m))

In [19]:
novel_compounds=pd.DataFrame(novel_compounds_list)
novel_compounds[['modifier','head']]=novel_compounds[0].str.split(" ",expand=True)
novel_compounds.drop(0,axis=1,inplace=True)
novel_compounds

,modifier,head
0,#,husky
1,#,nat
2,#,new
3,#,region
4,#,text
...,...,...
12567,•,switzerland
12568,•,transportation
12569,•,type
12570,•,water


In [20]:
positive_heads=pd.merge(novel_compounds.drop('modifier',axis=1),heads,on=["head"],how='left')

positive_modifiers=pd.merge(novel_compounds.drop('head',axis=1),modifiers,on=["modifier"])
#positive_df['Plausibility']=True

In [21]:
to_remove=positive_heads[positive_heads.isna().any(axis=1)].index.values[0]
to_remove

10099

In [22]:
positive_modifiers.drop(to_remove,inplace=True)
positive_heads.drop(to_remove,inplace=True)
novel_compounds.drop(to_remove,inplace=True)

In [23]:
assert positive_modifiers.shape[0]==positive_heads.shape[0]==novel_compounds.shape[0]

In [24]:
positive_heads_tensor = torch.tensor(positive_heads.drop('head',axis=1).values)
positive_heads_tensor.shape

torch.Size([12571, 400])

In [25]:
positive_modifiers_tensor = torch.tensor(positive_modifiers.drop('modifier',axis=1).values)
positive_modifiers_tensor.shape

torch.Size([12571, 400])

In [26]:
positive_Y=torch.ones(positive_modifiers_tensor.shape[0])
positive_Y.shape

torch.Size([12571])

In [27]:
positive_class=torch.cat((positive_modifiers_tensor, positive_heads_tensor), 1)
positive_class.shape

torch.Size([12571, 800])

In [28]:
def neg_df_creator(file):
    pkl_file=pkl.load( open(file,'rb'))
    df=pd.DataFrame(pkl_file)
    
    df.columns=['modifier','head']
    negative_heads=pd.merge(df.drop('modifier',axis=1),heads,on=["head"],how='left')
    negative_modifiers=pd.merge(df.drop('head',axis=1),modifiers,on=["modifier"],how='left')
    
    to_remove_mods=negative_modifiers[negative_modifiers.isna().any(axis=1)].index.to_list()
    to_remove_heads=negative_heads[negative_heads.isna().any(axis=1)].index.to_list()
    
    negative_modifiers.drop(to_remove_mods+to_remove_heads,inplace=True)
    negative_heads.drop(to_remove_mods+to_remove_heads,inplace=True)
    negative_heads_tensor = torch.tensor(negative_heads.drop('head',axis=1).values)
    negative_modifiers_tensor = torch.tensor(negative_modifiers.drop('modifier',axis=1).values)

    negative_Y=torch.zeros(negative_modifiers_tensor.shape[0])
    negative_class=torch.cat((negative_modifiers_tensor, negative_heads_tensor), 1)

    return negative_class,negative_Y

In [29]:
def tensor_joiner(files):
    tensor_list=[]
    for file in files:
        negative_class,negative_Y=neg_df_creator(file)
        X=torch.cat((positive_class, negative_class), 0)
        Y=torch.cat((positive_Y,negative_Y),0)
        tensor_list.append([X,Y])
    
    return tensor_list

In [31]:
corrupt_modifier_files=[]
for file in glob.glob("/data/dharp/compounds/datasets/corrupt_modifier*"):
    corrupt_modifier_files.append(file)
corrupt_modifiers=tensor_joiner(corrupt_modifier_files)

In [32]:
corrupt_head_files=[]
for file in glob.glob("/data/dharp/compounds/datasets/corrupt_head*"):
    corrupt_head_files.append(file)
corrupt_heads=tensor_joiner(corrupt_head_files)

In [33]:
input_size = 800
hidden_size = 300
num_classes = 2
num_epochs = 100
batch_size = 100
learning_rate = 0.001

In [34]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes,bias=False)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

model = NeuralNet(input_size, hidden_size, num_classes).cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

In [35]:
def looper(datasets):
    total_accuracy=[]
    
    for dataset in datasets:
        X=dataset[0]
        Y=dataset[1]
        n = len(X)  # how many total elements you have
        n_test = int( n * .2 )  # number of test/val elements
        n_train = n - n_test
        idx = list(range(n))  # indices to all elements
        random.shuffle(idx)  # in-place shuffle the indices to facilitate random splitting
        train_idx = idx[:n_train]
        test_idx = idx[n_train:]
        trX=X[train_idx].float().to(device)
        teX=X[test_idx].float().to(device)

        trY=Y[train_idx].long().to(device)
        teY=Y[test_idx].long().to(device)
        model = NeuralNet(input_size, hidden_size, num_classes).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        n_examples=trX.shape[0]
        for i in range(num_epochs):

            cost = 0.

            num_batches = n_examples // batch_size
            for k in range(num_batches):
                start, end = k * batch_size, (k + 1) * batch_size
                outputs = model(trX[start:end])
                loss = criterion(outputs, trY[start:end])

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            #if (k+1) % 100 == 0:
            #print ('Epoch [{}/{}], Loss: {:.4f}'.format(i+1, num_epochs, loss.item()))

        with torch.no_grad():
            correct=0
            total=0
            outputs = model(teX)
            _, predicted = torch.max(outputs.data, 1)
            total += teY.size(0)
            correct += (predicted == teY).sum().item()
        curr_acc=100 * correct / total
        print(curr_acc)
        total_accuracy.append(curr_acc)
    return total_accuracy
        

In [36]:
cor_head_acc=looper(corrupt_heads)

71.03324706350786
74.25841130798328
70.70646766169155
72.28747760302608
71.29777070063695
71.78976707147123
72.92993630573248
71.92912602030658
70.23691021302011
72.3328025477707


In [37]:
cor_mod_acc=looper(corrupt_modifiers)

68.14564265817748
68.95522388059702
70.0358137684043
68.65047770700637
66.92537313432835
69.29964186231595
69.13432835820896
69.01492537313433
69.2139303482587
69.45881416633506


82.91589770630108
83.12681254943317
82.59952544160295
82.76649969241585
83.14438878636084
83.05650760172247
82.90710958783724
83.22348185253537
82.67861850777749
82.74892345548818

In [38]:
round(np.mean(cor_head_acc),2)

71.88

In [39]:
round(np.std(cor_head_acc),2)

1.11

82.88074523244573
83.40803234027595
82.29194129536866
81.87889972756832
82.72255910009667
82.21284822919412
82.45891554618156
82.5643729677476
82.6610422708498
82.3007294138325

In [40]:
round(np.mean(cor_mod_acc),2)

68.88

In [41]:
round(np.std(cor_mod_acc),2)

0.8